In [10]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from collections import deque

In [11]:
# Load in the data 
def load_stock_csv():
    sp = pd.read_csv('stock_prices.csv', header=[0,1])
    sp.set_index(sp.columns[0], inplace= True)
    sp.index.name = 'Date'
    sp.columns.names =['Metric', 'Stock_name']
    return  sp.iloc[1:, :]

df = load_stock_csv()

def add_stock_price(stock, sp = df):
    out = sp.xs(key = stock.upper(), axis = 1, level = 'Stock_name', drop_level=True)
    out.columns.names = ['']
    out.columns = out.columns.get_level_values(0)
    return out.copy()


STOCKS = df.columns.get_level_values(1).unique().to_list()
print('Stocks Loaded: ', STOCKS)


Stocks Loaded:  ['AAPL', 'BNO', 'CORN', 'FDN', 'GDX', 'IBB', 'IGV', 'ITB', 'JETS', 'LQD', 'MOO', 'PBW', 'PHO', 'SMH', 'SPY', 'TAN', 'TLT', 'UUP', 'WEAT', 'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XRT']


# Make some targets for the model to predict with 
- target is the variable 
- seq_len is how many days we will use to predict 
- future_pred is the amount of days we will try to predict into the future. 

    - The classify clause allows you to make a dummy column indicating a buy or a sell signal 
    - when the future price is greater than the current price, this is a buy signal. 
    - when the future price is less than the current price, this is a sell or neutral signal. 

# Need to seperate out of sample data.
- good practice for time series
- without doing this the model will overfit, causing the loss to skyrocket. 
- seperating the last 5% of the data allows us to forward test on the last 5 percent. 

In [12]:
def add_stock_price(stock, sp = df):
    out = sp.xs(key = stock.upper(), axis = 1, level = 'Stock_name', drop_level=True).dropna()
    out.columns.names = ['']
    out.columns = out.columns.get_level_values(0)
    return out.copy()
    
def classify(current, future):
    """ Classify a Buy or Sell Signal, If the future price is greater than the current price, then it is a buy signal, otherwise it is a sell signal"""
    if float(future) >float(current):
        return 1
    else:
        return 0

def shift_target(stock, future_pred, target = 'Close'):
    """Shift the Target variable into the future"""
    df = add_stock_price(stock)
    df['future'] = df[target].shift(-future_pred) # Shifting the Price Variable, (From the most recent) 
    df['target'] = list(map(classify,df[target],df['future'])) # Classifying Current Price with Future Price (Lagged Vs. Current)
    return df.copy()

def split_data(stock, val_size = .05, future_pred = 6):
    df = shift_target(stock, future_pred)
    time = sorted(df.index.values) # Sorting the Date values. 
    last5 = time[-int(val_size *len(time))] # Last 5% of the data Used for Validation
    val_df = df[df.index>=last5] # Validation Data
    df = df[df.index<=last5] # Training Data
    return df, val_df

# Normalizing data. 
- normalize the data
- create the sequences we want to use to predict on
- balance the dataset 

In [13]:
def prep_df(df, seq_len = 30, target = 'target'):
    """Prepare the dataframe for the LSTM model, seq_length is the number of days we will use to predict the next days specified with future_pred"""
    df = df.drop(['future', 'Volume'],axis=1)
    
    for col in df.columns:
        if col != "target":
            df[col] = df[col].pct_change() # gets the percent change 
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col].values) # Scaling the data. 
    df.dropna(inplace=True)

    seq_data = []
    prev_days = deque(maxlen = seq_len) # appends all the data to the empty list, as it gets new items it deletes the old items 

    for i in df.values:
        prev_days.append([n for n in i[:-1]]) # each of the columns up to the last column (target)
        if len(prev_days) == seq_len:
            seq_data.append([np.array(prev_days), i[-1]])
    
    # Randomizing the list order of seq_data, so that the model does not get biased towards the first few days of the data.
    np.random.shuffle(seq_data)

    # balance the data 
    buys = []
    sells = []
    for seq, target in seq_data:
        if target== 0:
            sells.append([seq,target])
        elif target ==1:
            buys.append([seq, target])

    np.random.shuffle(buys)
    np.random.shuffle(sells)

    lower = min(len(buys), len(sells))
    buys = buys[:lower]
    sells = sells[:lower]

    seq_data = buys+sells
    np.random.shuffle(seq_data)

    X = []
    y = []
    
    for seq, target in seq_data:
        X.append(seq)
        y.append(target)
    X, y = np.array(X),np.array(y)
    print('Train Shape: ', X.shape, 'Target Shape: ', y.shape)
    return(X,y)

def prep_train_val(stock, seq_len = 30, future_pred = 6, target = 'target', val_size = .05):
    df, val_df = split_data(stock, val_size, future_pred)
    X_train, y_train = prep_df(df, seq_len)
    X_val, y_val = prep_df(val_df, seq_len)
    return X_train, y_train, X_val, y_val

In [14]:
# Shift Data and show dataframe
from IPython.display import display
data_df = shift_target('bno', 5)
display(data_df.head(), data_df.tail())

# Splitn data and Print Length of each 
train_df, val_df = split_data('bno')
print(f'Training df: {len(train_df)}')
print(f'Validation df: {len(val_df)}')

# Note that the Predictors are an array of arrays of arrays, where each array is a sequence of the previous days.
print('\nActual Data Prep\n')
X_train, y_train, X_val, y_val = prep_train_val('bno', seq_len = 30, future_pred = 6, target = 'target', val_size = .05)
print(f'train data: {len(X_train)} validation: {len(X_val)}')


,Adj Close,Close,High,Low,Open,Volume,future,target
Date,,,,,,,,
2010-06-02,25.590000,25.590000,25.615000,25.320000,25.485001,26800.0,25.330000,0
2010-06-03,25.830000,25.830000,25.915001,25.420000,25.715000,66800.0,25.875000,1
2010-06-04,24.700001,24.700001,25.245001,24.670000,25.245001,12000.0,25.559999,1
2010-06-07,24.795000,24.795000,24.985001,24.565001,24.799999,29000.0,25.639999,1
2010-06-08,24.690001,24.690001,24.850000,24.594999,24.695000,65400.0,26.160000,1


,Adj Close,Close,High,Low,Open,Volume,future,target
Date,,,,,,,,
2022-08-08,30.160000,30.160000,30.250000,29.490000,29.520000,208400.0,NaN,0
2022-08-09,30.129999,30.129999,30.740000,29.870001,30.420000,167400.0,NaN,0
2022-08-10,30.320000,30.320000,30.639999,29.280001,30.010000,376800.0,NaN,0
2022-08-11,31.070000,31.070000,31.309999,30.559999,30.860001,333800.0,NaN,0
2022-08-12,30.660000,30.660000,30.799999,30.309999,30.719999,223400.0,NaN,0


Training df: 2910
Validation df: 153

Actual Data Prep

Train Shape:  (2684, 30, 5) Target Shape:  (2684,)
Train Shape:  (116, 30, 5) Target Shape:  (116,)
train data: 2684 validation: 116


# Creating the model 

In [36]:
import datetime as dt
from venv import create 
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense,Dropout,LSTM,BatchNormalization
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report,confusion_matrix
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

def create_folder(stock, seq_len, future_pred):
# Save the data to csv in their own folder with a date time stammp 
    outname = f'{stock}-stock-{seq_len}-seq-{future_pred}=pred-{dt.time()}'
    outdir = f'logs/'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname)
    return fullname

def rnn_predictor(stock, seq_len = 28, future_pred = 6,  val_size = .10 , epoch = 10, batch_size = 64):
    """ 
    1. Create a RNN Model given a stock that is in the data loaded. 
     - use seq length to specify how many days are used to make the next number of predictions specified with future_pred
     - valiadtion size is the percentage of the data used for validation 
     - epoch is the number of epochs used for training
     - batch size is the number of data points used for each epoch
    """
    # Data Preperation 
    train_x, train_y, val_x, val_y = prep_train_val(stock, seq_len, future_pred, val_size)

    # Tensorflow Model 
    model = Sequential()
    model.add(tf.keras.layers.LSTM(128,input_shape=(train_x.shape[1:]), return_sequences=False))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model = Sequential()
    model.add(tf.keras.layers.LSTM(128,input_shape=(train_x.shape[1:]), return_sequences=False))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())

    model = Sequential()
    model.add(tf.keras.layers.LSTM(128,input_shape=(train_x.shape[1:])))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(2, activation='softmax'))

    opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)
    model.compile(loss= 'sparse_categorical_crossentropy',optimizer= opt, metrics = ['accuracy'])

    # callbacks 
    tensorboard = TensorBoard(log_dir=create_folder(stock, seq_len, future_pred))
    filepath = "RNN_FINAL-{epoch:02d}-{val_accuracy:.3f}"
    checkpoint = ModelCheckpoint('models/{}.h5'.format(filepath, monitor='val_accuracy', save_best_only= True, mode= 'max'))

    history = model.fit(
        train_x,train_y, 
        batch_size= batch_size,
        epochs = epoch,
        validation_data= (val_x, val_y),
        callbacks=[tensorboard,checkpoint]
        )
    
    
    predictions = np.argmax(model.predict(val_x), axis=1)
    print(classification_report(val_y,predictions))
    return history, predictions


In [37]:
print(STOCKS)

['AAPL', 'BNO', 'CORN', 'FDN', 'GDX', 'IBB', 'IGV', 'ITB', 'JETS', 'LQD', 'MOO', 'PBW', 'PHO', 'SMH', 'SPY', 'TAN', 'TLT', 'UUP', 'WEAT', 'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XRT']


In [39]:
seq_len = 10
future_pred = 3

d = {}

for x in ['SPY', 'TLT', 'XME', 'AAPL']: 
    print('Stock: ', x)
    history, predictions = rnn_predictor(x, seq_len, future_pred, val_size = .05 , epoch = 13, batch_size = 128)
    d[x] = [history, predictions]



Stock:  SPY
Train Shape:  (5634, 10, 5) Target Shape:  (5634,)
Train Shape:  (294, 10, 5) Target Shape:  (294,)


2022-08-16 02:15:03.113753: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-08-16 02:15:03.113773: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-08-16 02:15:03.113839: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


Epoch 1/13


2022-08-16 02:15:03.841235: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-08-16 02:15:04.102308: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-08-16 02:15:05.184439: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


 2/45 [>.............................] - ETA: 6s - loss: 0.7871 - accuracy: 0.4961  

...

48/48 [==============================] - ETA: 0s - loss: 0.7733 - accuracy: 0.4998

2022-08-16 02:16:34.286514: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-08-16 02:16:34.439712: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


48/48 [==============================] - 7s 109ms/step - loss: 0.7733 - accuracy: 0.4998 - val_loss: 0.6936 - val_accuracy: 0.5032
Epoch 2/13
48/48 [==============================] - 2s 46ms/step - loss: 0.7233 - accuracy: 0.5127 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/13
48/48 [==============================] - 2s 44ms/step - loss: 0.7106 - accuracy: 0.5199 - val_loss: 0.6943 - val_accuracy: 0.5000
Epoch 4/13
48/48 [==============================] - 2s 44ms/step - loss: 0.7076 - accuracy: 0.5158 - val_loss: 0.6930 - val_accuracy: 0.5000
Epoch 5/13
48/48 [==============================] - 2s 43ms/step - loss: 0.7025 - accuracy: 0.5257 - val_loss: 0.6935 - val_accuracy: 0.4903
Epoch 6/13
48/48 [==============================] - 2s 43ms/step - loss: 0.7005 - accuracy: 0.5214 - val_loss: 0.6950 - val_accuracy: 0.4935
Epoch 7/13
48/48 [==============================] - 2s 44ms/step - loss: 0.6942 - accuracy: 0.5252 - val_loss: 0.6942 - val_accuracy: 0.4805
Epoch 8/13
48/48 [=====

2022-08-16 02:17:00.827438: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-08-16 02:17:00.895357: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


              precision    recall  f1-score   support

         0.0       0.51      0.47      0.49       154
         1.0       0.51      0.55      0.52       154

    accuracy                           0.51       308
   macro avg       0.51      0.51      0.51       308
weighted avg       0.51      0.51      0.51       308



In [40]:
#%tensorboard --logdir=logs/

In [42]:
sorted([(x, max(d[x][0].history['val_accuracy'])) for x in ['SPY', 'TLT', 'XME', 'AAPL']], key=lambda x: x[1], reverse=True)[:5]

[('SPY', 0.5408163070678711),
 ('XME', 0.529411792755127),
 ('TLT', 0.5247524380683899),
 ('AAPL', 0.5162337422370911)]

In [43]:
d['SPY'][1]

array([1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 0, 0, 1, 1])